In [ ]:
%pkg load
clearvars;
clear all;
close all;
pkg load image
%pkg load mathgeom %(package needs to be installed from the https://octave.sourceforge.io/matgeom/ )

tic;
% load image file
%img = imread('stains.png');
display_limit=2000; % maximum diagonal length of image that computer can display
measure_angles=false;

%img=imread('DSC_0130.png'); %540x700 pixels
img=imread('HP_5.jpg'); % 16K by 27k pixels color
%img=imread('HP_5.png'); % 16K by 27k pixels BW threshold


% convert image to grayscale if image is not grayscale yet
if isequal(img(:,:,1),img(:,:,2),img(:,:,3)) % if image is fake 3 color but already thresholded
  img=img(:,:,1);
  binary_img=~img;

else

  gray_img = rgb2gray(img); % if image needs to be thresholded


  % apply threshold to image to create binary image
  threshold = 0.5;
  binary_img = imcomplement(im2bw(gray_img, threshold));
  warning('end calculation binary image');toc;
end


% compute properties of connected regions in binary image
region_props = regionprops(binary_img,  'Centroid');
a=warning('end regionprops');toc;

%%shows image if image not too large
diagonal_image = (size(binary_img,1)^2+size(binary_img,2)^2)^(0.5);
if diagonal_image <display_limit %display input image and thresholded image since image is not too large
figure(1);clf;imshow(img);
figure(2);clf; imshow(binary_img);hold on;
figure(3);imshow(binary_img);

warning('end image show binary');toc;
else
warning('input image and thresholded image saved to file but not displayed because image too large');
end

if measure_angles==true;
  %% measure angles and display angles
else
  %% do nothing
endif

% save images
imwrite (binary_img,"output_binary_image.png");



toc;


In [ ]:
%plot -b inline
%plot -f svg

In [ ]:
clc; clf;
clear all;
graphics_toolkit("gnuplot")

%Inputs case 4
%=====
Humidity = 0.32;
Diff = 2.61e-5; % Diffusion coefficient, water vapor in air, m^2/s
csat = 0.023; % kg/m^3
V0 = 6.4;  % muL %  1 muL = 1e-9 m^3
rho = 1000; % kg/m^3
dt = 10;  % e.g. 10 s for 1 muL
Wetting_angle = 50; %degrees
%======

V(1) = V0*1e-9;  % convert to m^3
%v0 = 0; %airflow velocity in m/s
%====

AngRad(1) = Wetting_angle.*pi/180;
HbyR  =  tan(AngRad(1)/2)  ;
VnL = V(1)*1e12;
Rum = ((4.*VnL)./(pi.*HbyR.*(2+HbyR.^2))).^(1/3)*100; %in um
R(1)  = Rum*1e-6;
time(1)= 0;

for itime = 1:1:10000;

    if(AngRad(itime) > Wetting_angle*pi/180)
        CCR = 1;
        CCA = 0;
    else
        CCR = 0;
        CCA = 1;
    end

    if(AngRad(itime) < 1.57)
        mdotHuLarson     = pi*Diff*csat*(1-Humidity).*R(itime).*(0.27.*AngRad(itime).^2+1.30);
        mdot = mdotHuLarson;
    else
        mdotHyrophobic = Diff*csat*(1-Humidity).*R(itime).*tan(AngRad(itime)/2).*(6.11*exp(-0.45.*AngRad(itime).^2)+4.28);
        mdot = mdotHyrophobic;
    end

   %mdotConv = 0.6*(v0*R(itime)/Diff)^0.5;
    mdotConv = 0;

   time(itime+1) = time(itime)+dt;
   V(itime+1) = V(itime) - (mdot + mdotConv)*dt/rho;


   if(CCR==1)
   R(itime+1) = R(itime);
   myfun = @(x,c,d) x^3 + c*x - d;   % parameterized function
   c = 3*R(itime+1)^2;                    % parameter
   d = 6*V(itime+1)/pi;
   fun = @(x) myfun(x,c,d);    % function of x alone
   x = fzero(fun,0.1);
   AngRad(itime+1) = 2*atan(x/R(itime+1));
   end

    if(CCA==1)
    AngRad(itime+1) = AngRad(itime);
    th1 = AngRad(itime+1);
    prefac1 = sin(th1)*(2+cos(th1))/((1+cos(th1))^2);
    R(itime+1) = (3*V(itime+1)/(pi*prefac1 ))^0.3333;
    end

   ChangeVolPer = (V(1) - V(itime))/V(1) *100;
   %itime

   if(ChangeVolPer >= 99)
       itime
       break;
   end

end

%ChangeVolPer
%time(itime)

figure(1); clf;

plot(time,V*1e9,'-*','MarkerSize',0.8);
xlabel('Time(s)')
ylabel('V (\mu L)')
hold on;

figure(2); clf;

plot(time,AngRad*180/pi,'-*','MarkerSize',0.8);
xlabel('Time(s)')
ylabel('\theta (degrees)')
hold on;
figure(3); clf;

plot(time,2*R*1000,'-*','MarkerSize',0.8);
xlabel('Time(s)')
ylabel('Wetted diameter (mm)')
hold on;

In [ ]:
% Cell above needed to display plots properly
pkg load io
graphics_toolkit("gnuplot")

% load data

file_path = which("input.csv");
A=xlsread(file_path);

x=A(:,1);
y=A(:,2);

figure(1);clf
plot(x,y,'r+');
z=sin(x+y);
figure(2);clf
plot3(x,y,z)